In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):\
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing all the Dependencies**

In [2]:
import tensorflow
from tensorflow import keras
from keras.datasets import cifar10
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Loading Dataset**

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
X_train.shape

(50000, 32, 32, 3)

# **Normalizing**

In [6]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [7]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# **Data augmentation**

In [8]:
datagen = ImageDataGenerator(
    rotation_range=15,      # Rotate the image by up to 15 degrees
    width_shift_range=0.1,  # Shift the image horizontally by up to 10% of the width
    height_shift_range=0.1, # Shift the image vertically by up to 10% of the height
    horizontal_flip=True    # Randomly flip the image horizontally
)

# Fit the datagen on the training data
datagen.fit(X_train)

# Example of using the data generator
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# Check the shapes of the datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 10)


# **Hyperparameter Tunning**

In [10]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LeakyReLU
import tensorflow as tf

def build_model(hp):
    model = Sequential()
    
    # Convolutional layers
    for i in range(hp.Int("num_conv_layers", 1, 3)):
        filters = hp.Int(f"filters_{i}", min_value=16, max_value=128, step=16)
        kernel_size = hp.Choice(f"kernel_size_{i}", values=[3, 5])
        activation = hp.Choice(f"activation_{i}", values=["relu", "leakyrelu"])

        if activation == "leakyrelu":
            model.add(Conv2D(filters=filters, kernel_size=kernel_size, padding='same'))
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding='same'))

        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    # Fully connected layers
    for i in range(hp.Int("num_dense_layers", 1, 3)):
        units = hp.Int(f"units_{i}", min_value=8, max_value=128, step=8)
        activation = hp.Choice(f"activation_{i}", ["relu", "leakyrelu"])

        if activation == "leakyrelu":
            model.add(Dense(units=units))
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(Dense(units=units, activation=activation))

    # Output layer
    model.add(Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [11]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials=5,
    directory = "my_dir",
    project_name="tuner_CNN"
)

In [ ]:
tuner.search(
    train_generator, epochs=5, validation_data=(X_test, y_test)
)

Trial 4 Complete [00h 03m 02s]
val_accuracy: 0.6460000276565552

Best val_accuracy So Far: 0.6725000143051147
Total elapsed time: 00h 12m 10s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
2                 |3                 |num_conv_layers
64                |128               |filters_0
3                 |3                 |kernel_size_0
relu              |leakyrelu         |activation_0
3                 |2                 |num_dense_layers
8                 |48                |units_0
80                |80                |filters_1
5                 |3                 |kernel_size_1
leakyrelu         |leakyrelu         |activation_1
80                |16                |filters_2
5                 |5                 |kernel_size_2
leakyrelu         |leakyrelu         |activation_2
40                |64                |units_1
96                |112               |units_2

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 47s 28ms/step - accuracy: 0.1846 - l

In [12]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'num_conv_layers': 2, 'filters_0': 64, 'kernel_size_0': 5, 'activation_0': 'relu', 'num_dense_layers': 2, 'units_0': 72, 'filters_1': 112, 'kernel_size_1': 5, 'activation_1': 'leakyrelu', 'filters_2': 128, 'kernel_size_2': 3, 'activation_2': 'relu', 'units_1': 40}


# **Final Model**

In [33]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=112, kernel_size=5, padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

    # First Dense Layer
model.add(Dense(128, activation='relu'))
model.add(Dense(72, activation='relu'))
model.add(Dense(40))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(10, activation='softmax'))

In [34]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 64)     │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 112)    │       179,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 16, 16, 112)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 8, 8, 112)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 7168)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │       917,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 72)             │         9,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 40)             │         2,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │           410 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,114,426 (4.25 MB)

 Trainable params: 1,114,426 (4.25 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:

from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor = "val_loss",
    min_delta = 0.00001,
    patience = 15,
    verbose = 1,
    mode='auto',
    baseline = None,
    restore_best_weights = None
)

In [36]:
history= model.fit(train_generator, epochs=50,callbacks=callback, validation_data=(X_test, y_test))

Epoch 1/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 26ms/step - accuracy: 0.3336 - loss: 1.8062 - val_accuracy: 0.5359 - val_loss: 1.2749
Epoch 2/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 74s 22ms/step - accuracy: 0.5401 - loss: 1.2854 - val_accuracy: 0.6038 - val_loss: 1.0932
Epoch 3/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.5990 - loss: 1.1288 - val_accuracy: 0.6498 - val_loss: 1.0025
Epoch 4/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.6336 - loss: 1.0301 - val_accuracy: 0.6726 - val_loss: 0.9547
Epoch 5/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 21ms/step - accuracy: 0.6575 - loss: 0.9766 - val_accuracy: 0.6728 - val_loss: 0.9357
Epoch 6/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 21ms/step - accuracy: 0.6809 - loss: 0.9128 - val_accuracy: 0.7077 - val_loss: 0.8344
Epoch 7/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.6935 - loss: 0.8720 - val_accuracy: 0.7168 - val_loss: 0.8104
Epoch 8/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.7087 -

In [38]:
# Save the model
model.save("cnn_model_optimized.h5")

In [47]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load the model (if saved)
# model = load_model('path_to_your_model.h5')

# Function to preprocess the image
def preprocess_image(image_path):
    # Load the image with target size (32, 32)
    image = load_img(image_path, target_size=(32, 32))
    # Convert the image to array
    image = img_to_array(image)
    # Normalize the image
    image = image / 255.0
    # Expand dimensions to match the input shape of the model (1, 32, 32, 3)
    image = np.expand_dims(image, axis=0)
    return image

# Function to predict the class of the image
def predict_image_class(image_path, model):
    # Preprocess the image
    processed_image = preprocess_image(image_path)
    # Make predictions
    predictions = model.predict(processed_image)
    # Get the predicted class
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0], predictions

# Example usage
image_path = '/kaggle/input/cat-pik/cat.jpeg'  # Replace with your image path
predicted_class, predicted_probabilities = predict_image_class(image_path, model)

print(f"Predicted class: {predicted_class}")
print(f"Predicted probabilities: {predicted_probabilities}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted class: 5
Predicted probabilities: [[1.9329491e-06 2.3865754e-08 2.3763407e-05 8.0999419e-02 1.5848438e-03
  9.1541570e-01 9.0884983e-05 1.8819976e-03 1.3057179e-06 1.1748256e-07]]
